<a href="https://colab.research.google.com/github/ufrpe-ensino/ia-aulas/blob/master/00-intro/03_Visualizacao-Estatisticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualização de Dados e Estatísticas Descritivas em Python

Nesta aula, vamos aprender como explorar, descrever e visualizar dados utilizando Python, com foco em estatísticas descritivas e gráficos. Usaremos o dataset de imóveis do Recife.

## Objetivos
- Compreender o papel da visualização na análise de dados
- Calcular estatísticas descritivas básicas
- Gerar gráficos para explorar dados

## Importando bibliotecas

In [ ]:
!pip install seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Carregando o conjunto de dados
Vamos utilizar um conjunto de dados de imóveis do Recife disponível online.

In [ ]:
houses = pd.read_csv("https://raw.githubusercontent.com/ProfLuciano/intro_cd/gh-pages/data/recife.csv", encoding = "ISO-8859-1")
houses.head()

## Estatísticas Descritivas
Vamos calcular algumas estatísticas básicas para entender o conjunto de dados.

In [ ]:
houses.shape  # Dimensões do data frame

In [ ]:
houses.dtypes  # Tipos das colunas

### Estatísticas gerais

In [ ]:
houses.describe()  # Estatísticas gerais

In [ ]:
# Média, mediana, moda, variância e desvio padrão do preço
print('Média:', houses['price'].mean())
print('Mediana:', houses['price'].median())
print('Moda:', houses['price'].mode()[0])
print('Variância:', houses['price'].var())
print('Desvio padrão:', houses['price'].std())

### Estatísticas da área

In [ ]:
houses['area'].describe()

## Correlações

In [ ]:
houses.columns

## Correlação de Pearson

A correlação de Pearson é uma medida estatística que avalia a força e a direção da relação linear entre duas variáveis quantitativas. Seu coeficiente, conhecido como **r de Pearson**, varia de -1 a 1:

- **r = 1**: correlação linear positiva perfeita (quando uma variável aumenta, a outra também aumenta proporcionalmente).
- **r = -1**: correlação linear negativa perfeita (quando uma variável aumenta, a outra diminui proporcionalmente).
- **r = 0**: ausência de correlação linear.

Valores próximos de 1 ou -1 indicam forte correlação, enquanto valores próximos de 0 indicam correlação fraca ou inexistente. É importante lembrar que a correlação de Pearson só captura relações lineares e não implica causalidade entre as variáveis.

No contexto da análise de dados de imóveis, a correlação de Pearson pode ser usada, por exemplo, para investigar se existe relação entre o preço dos imóveis e características como área, número de quartos ou banheiros.

In [ ]:
pearson_corr = houses.loc[houses.operation=='sell',
           ['price', 'bedrooms', 'area',
            'bathrooms', 'ensuites']].corr(method='pearson')
pearson_corr

In [ ]:
sns.heatmap(pearson_corr, annot=True, cmap='coolwarm', center=0)

### Correlação de Spearman

A correlação de Spearman é uma medida estatística que avalia a relação monotônica entre duas variáveis quantitativas ou ordinais. Diferente da correlação de Pearson, que mede apenas relações lineares, a correlação de Spearman é baseada nos postos (ranks) dos dados e pode capturar relações não-lineares, desde que sejam monotônicas (ou seja, que uma variável aumente ou diminua consistentemente em relação à outra).

O coeficiente de Spearman varia de -1 a 1:
- **1** indica uma correlação monotônica positiva perfeita.
- **-1** indica uma correlação monotônica negativa perfeita.
- **0** indica ausência de correlação monotônica.

A correlação de Spearman é especialmente útil quando os dados apresentam outliers ou não seguem uma distribuição normal, sendo uma alternativa robusta à correlação de Pearson em muitos cenários de análise de dados. No contexto dos imóveis, pode ser usada para investigar relações entre preço, área, número de quartos, banheiros, entre outros, mesmo quando essas relações não são estritamente lineares.

In [ ]:
spearman_corr = houses.loc[houses.operation=='sell',
           ['price', 'bedrooms', 'area',
            'bathrooms', 'ensuites']].corr(method='spearman')
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', center=0)

## Visualização de Dados
### Histograma do preço dos imóveis

In [ ]:
(houses['price']/1000).hist(bins=30, figsize=(10,5))
plt.title('Distribuição dos Preços (milhares)')
plt.xlabel('Preço (milhares)')
plt.ylabel('Frequência')
plt.show()

### Definindo se um imóvel está à venda ou para locação a partir do preço

In [ ]:
# Adicionando coluna para aluguel e venda
houses['operation'] = 'sell'
houses.loc[(houses['price'] > 100) & (houses['price'] < 30000),'operation'] = 'rent'
houses.head()

### Boxplot da área dos imóveis

In [ ]:
houses.boxplot(column=['area'])
plt.title('Boxplot da Área dos Imóveis')
plt.ylabel('Área (m²)')
plt.show()

### Gráfico de barras: quantidade de imóveis por tipo

In [ ]:
houses['type'].value_counts().plot(kind='bar', figsize=(8,4))
plt.title('Quantidade de Imóveis por Tipo')
plt.xlabel('Tipo')
plt.ylabel('Contagem')
plt.show()

### Gráfico de dispersão: área vs preço

In [ ]:
houses.plot.scatter(x='area', y='price', alpha=0.5)
plt.title('Área vs Preço')
plt.xlabel('Área (m²)')
plt.ylabel('Preço')
plt.show()

### Visualizando dados com geolocalização

In [ ]:
!pip install folium

In [ ]:
import folium

# Centro aproximado de Recife
mapa = folium.Map(location=[-8.05, -34.9], zoom_start=12)

for _, row in houses.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='blue' if row['operation'] == 'sell' else 'green',
        fill=True,
        fill_opacity=0.6,
        popup=f"Preço: R${row['price']}\nTipo: {row['type']}"
    ).add_to(mapa)

mapa

### Heat map dos preços de venda

In [ ]:
from folium.plugins import HeatMap

# Filtra imóveis para aluguel com latitude, longitude e preço válidos
rent_houses = houses[(houses['operation'] == 'sell') &
                     houses['latitude'].notnull() &
                     houses['longitude'].notnull() &
                     houses['price'].notnull()]

# Cria o mapa centralizado em Recife
heatmap_map = folium.Map(location=[-8.05, -34.9], zoom_start=12)

# Adiciona o HeatMap
HeatMap(
    data=rent_houses[['latitude', 'longitude', 'price']].values,
    radius=10,
    max_zoom=13
).add_to(heatmap_map)

heatmap_map

## Exercícios

1. Calcule o preço médio do metro quadrado de venda e aluguel para Recife.
2. Plote um histograma da coluna `area`.
3. Plote um boxplot do preço para apartamentos e casas separadamente.
4. Plote um gráfico de dispersão entre `bedrooms` e `area`.
5. Encontre os bairros com maior e menor valor de venda e aluguel do metro quadrado.
6. Encontre a variável que tem maior correlação de spearman com o preço de imóveis à venda

In [ ]:
# 1. Preço médio do metro quadrado de venda e aluguel

# 2. Histograma da área

# 3. Boxplot do preço para apartamentos e casas

# 4. Scatter plot entre bedrooms e area

# 5. Bairros com maior e menor valor de venda do metro quadrado